In [1]:
import cirq

# Define the qubits
q0, q1 = cirq.LineQubit.range(2)

# Create a circuit
circuit = cirq.Circuit()

# Apply a Hadamard gate to the first qubit
circuit.append(cirq.H(q0))

# Apply a CNOT gate with the first qubit as control and the second as target
circuit.append(cirq.CNOT(q0, q1))

# Measure the qubits
circuit.append(cirq.measure(q0, key='q0'))
circuit.append(cirq.measure(q1, key='q1'))

# Print the circuit
print("Circuit:")
print(circuit)

# Simulate the circuit
simulator = cirq.Simulator()
result = simulator.run(circuit, repetitions=1000)

# Print the results
print("\nResults:")
print(result.histogram(key='q0'))
print(result.histogram(key='q1'))

#If you have access to the Quokka device
#You would add something along these lines (replace with your actual device details):

#from cirq.google import Engine
#engine = Engine(project_id='your-project-id', location='your-location')
#device = engine.get_device(device_id='quokka')

#And then run on the device (after potentially transpiling the circuit for the device)
#result = engine.run(circuit, repetitions=100)


Circuit:
0: ───H───@───M('q0')───
          │
1: ───────X───M('q1')───

Results:
Counter({1: 510, 0: 490})
Counter({1: 510, 0: 490})


In [ ]:
# Task 1: Entanglement Demonstrations - Bell State using Cirq on Quokka Device
# Creating Bell state |ψ⟩ = (1/√2)(|00⟩ + |11⟩) with Cirq and running on Quokka

import cirq
import numpy as np
import requests
import json
from collections import Counter

# Step 1: Create the Bell State circuit using Cirq
print("=== Creating Bell State Circuit with Cirq ===")

# Define the qubits
q0, q1 = cirq.LineQubit.range(2)

# Create a circuit
circuit = cirq.Circuit()

# Apply a Hadamard gate to the first qubit (creates superposition)
circuit.append(cirq.H(q0))

# Apply a CNOT gate with the first qubit as control and the second as target
circuit.append(cirq.CNOT(q0, q1))

# Measure the qubits
circuit.append(cirq.measure(q0, key='q0'))
circuit.append(cirq.measure(q1, key='q1'))

# Print the circuit
print("Cirq Circuit:")
print(circuit)

# Step 2: Test locally first (simulate with Cirq)
print("\n=== Local Simulation with Cirq ===")
simulator = cirq.Simulator()
local_result = simulator.run(circuit, repetitions=100)

print("Local simulation results:")
print(f"q0 outcomes: {local_result.histogram(key='q0')}")
print(f"q1 outcomes: {local_result.histogram(key='q1')}")

# Step 3: Convert Cirq circuit to QASM for Quokka
print("\n=== Converting to QASM for Quokka ===")

def cirq_to_qasm_for_quokka(circuit):
    """
    Convert Cirq circuit to QASM format compatible with Quokka device
    """
    qasm_lines = []
    qasm_lines.append("OPENQASM 2.0;")
    
    # Find all qubits used in the circuit
    qubits = sorted(circuit.all_qubits())
    num_qubits = len(qubits)
    
    # Create qubit and classical registers
    qasm_lines.append(f"qreg q[{num_qubits}];")
    qasm_lines.append(f"creg c[{num_qubits}];")
    
    # Map Cirq qubits to indices
    qubit_map = {qubit: i for i, qubit in enumerate(qubits)}
    
    # Convert operations
    for moment in circuit:
        for operation in moment:
            if isinstance(operation.gate, cirq.HPowGate):
                # Hadamard gate
                qubit_idx = qubit_map[operation.qubits[0]]
                qasm_lines.append(f"h q[{qubit_idx}];")
            elif isinstance(operation.gate, cirq.CXPowGate):
                # CNOT gate
                control_idx = qubit_map[operation.qubits[0]]
                target_idx = qubit_map[operation.qubits[1]]
                qasm_lines.append(f"cx q[{control_idx}],q[{target_idx}];")
            elif isinstance(operation, cirq.MeasurementGate):
                # Measurement - handled separately below
                pass
    
    # Add measurements (assume measuring all qubits to corresponding classical bits)
    for i in range(num_qubits):
        qasm_lines.append(f"measure q[{i}] -> c[{i}];")
    
    return "\n".join(qasm_lines)

# Convert circuit to QASM
qasm_code = cirq_to_qasm_for_quokka(circuit)
print("Generated QASM code:")
print(qasm_code)

# Step 4: Send to Quokka device
print("\n=== Running on Quokka Device ===")
req_str_qasm = 'http://quokka1.quokkacomputing.com/qsim/qasm'

data = {
    'script': qasm_code,
    'count': 1000  # Run 1000 shots for good statistics
}

try:
    result = requests.post(req_str_qasm, json=data)
    json_obj = json.loads(result.content)
    
    print("Raw Quokka response:")
    print(json_obj)
    
    if json_obj['error'] == 'no error':
        # Step 5: Process and analyze results
        print("\n=== Processing Quokka Results ===")
        
        # Extract measurement results
        measurements = json_obj['result']['c']
        
        # Count the different outcomes
        outcome_counts = Counter()
        for measurement in measurements:
            # Convert [q0, q1] to binary string
            outcome = f"{measurement[0]}{measurement[1]}"
            outcome_counts[outcome] += 1
        
        print("Measurement outcomes from Quokka:")
        total_shots = len(measurements)
        for outcome, count in sorted(outcome_counts.items()):
            probability = count / total_shots
            print(f"|{outcome}⟩: {count}/{total_shots} ({probability:.3f} probability)")
        
        # Step 6: Verify Bell state properties
        print("\n=== Bell State Analysis ===")
        print("Expected for ideal Bell state |ψ⟩ = (1/√2)(|00⟩ + |11⟩):")
        print("- Only |00⟩ and |11⟩ outcomes should appear")
        print("- Each should have ~50% probability")
        
        # Check for perfect Bell state behavior
        expected_outcomes = ['00', '11']
        unexpected_outcomes = [outcome for outcome in outcome_counts.keys() 
                             if outcome not in expected_outcomes]
        
        if unexpected_outcomes:
            print(f"Unexpected outcomes detected: {unexpected_outcomes}")
            print("This indicates noise or decoherence in the quantum device")
        else:
            print("Only expected Bell state outcomes (|00⟩ and |11⟩) observed")
        
        # Calculate Bell state fidelity
        bell_outcomes = outcome_counts.get('00', 0) + outcome_counts.get('11', 0)
        fidelity = bell_outcomes / total_shots
        print(f"\nBell state fidelity: {fidelity:.3f}")
        
        if fidelity > 0.9:
            print("Excellent entanglement fidelity!")
        elif fidelity > 0.7:
            print("Good entanglement with some noise")
        else:
            print("Low fidelity - significant noise or errors present")
        
        # Compare ideal vs actual probabilities
        ideal_prob = 0.5
        actual_00 = outcome_counts.get('00', 0) / total_shots
        actual_11 = outcome_counts.get('11', 0) / total_shots
        
        print(f"\nProbability comparison:")
        print(f"|00⟩: Ideal = {ideal_prob:.3f}, Actual = {actual_00:.3f}")
        print(f"|11⟩: Ideal = {ideal_prob:.3f}, Actual = {actual_11:.3f}")
        
        # Step 7: Extract random bits for analysis
        print("\n=== Random Bit Generation ===")
        bits = list(np.concatenate(list(json_obj['result'].values())).flat)
        print(f"Generated {len(bits)} random bits from entangled measurements")
        print(f"First 20 bits: {bits[:20]}")
        
        # Optional: Test randomness (uncomment if you have randtest installed)
        # try:
        #     import randtest
        #     is_random = randtest.random_score(bits)
        #     print(f"Randomness test passed: {is_random}")
        # except ImportError:
        #     print("Install randtest with 'pip install randtest' for randomness analysis")
        
    else:
        print(f"Error from Quokka device: {json_obj['error']}")
        print(f"Error code: {json_obj['error_code']}")
        
except Exception as e:
    print(f"Connection error: {e}")
    print("Please check:")
    print("1. Internet connection")
    print("2. Quokka service availability")
    print("3. QASM code format")

print("\n=== Task 1 Summary ===")
print("✅ Created Bell state using Cirq: |ψ⟩ = (1/√2)(|00⟩ + |11⟩)")
print("✅ Converted Cirq circuit to QASM format")
print("✅ Successfully ran entangled state on Quokka quantum device")
print("✅ Analyzed quantum entanglement properties and fidelity")

=== Creating Bell State Circuit with Cirq ===
Cirq Circuit:
0: ───H───@───M('q0')───
          │
1: ───────X───M('q1')───

=== Local Simulation with Cirq ===
Local simulation results:
q0 outcomes: Counter({0: 53, 1: 47})
q1 outcomes: Counter({0: 53, 1: 47})

=== Converting to QASM for Quokka ===
Generated QASM code:
OPENQASM 2.0;
qreg q[2];
creg c[2];
h q[0];
cx q[0],q[1];
measure q[0] -> c[0];
measure q[1] -> c[1];

=== Running on Quokka Device ===
Raw Quokka response:
{'error': 'no error', 'error_code': 0, 'result': {'c': [[1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [0, 0], [1, 1], [1, 1], [1, 1], [0, 0], [0, 0], [1, 1], [0, 0], [0, 0], [1, 1], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [1, 1], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [1, 1], [0, 0], [0, 0], [0, 0], [1, 1], [0, 0], [1, 1], [0, 0], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [0, 0], [1, 1], [0, 0], [0, 0], [0, 0], [0, 0], [1, 1], [0, 0], [1, 1], [1, 1], [0, 0], [0, 0], [1, 1], [1, 1